In [3]:
import pandas as pd, numpy as np
import json

In [4]:
progress = pd.read_csv('historical_progresses_dec.csv')

In [5]:
progress[progress['subjectCode'] == 'en'].groupby(['subjectCode', 'studentName', 'lessonId']).agg({'score': [np.max], 'lessonOrder': lambda x: np.mean(x)})

score lessonOrder
                                    amax    <lambda>
subjectCode studentName lessonId                    
en          Aadharsha   en1600        21       146.0
                        en2000        29       204.0
                        en2001        59       205.0
                        en3100        24       290.0
                        en_PreQuiz    67         0.0
...                                  ...         ...
            vaisnavi    en0500        83        59.0
                        en0501       100        60.0
                        en0502        50        61.0
                        en_PreQuiz    42         0.0
                        pram         100        18.0

[12300 rows x 2 columns]

In [6]:
students = {}
for row in progress.itertuples():
    score = row[7]
    student = row[10]
    subject = row[12]
    lesson_order = row[14]
    date_time = row[13]
    if student not in students:
        students[student] = {subject: {lesson_order: (score, date_time)}}
    elif subject not in students[student]:
        students[student][subject] = {lesson_order: (score, date_time)}
    elif lesson_order not in students[student][subject]:
        students[student][subject][lesson_order] = (score, date_time)
    elif students[student][subject][lesson_order][0] < score:
        students[student][subject][lesson_order] = (score, date_time)


In [7]:
print(students)

{'KD07': {'kn': {117: (96, 'Tue Dec 14 2021 12:23:18 GMT+0530 (India Standard Time)'), 118: (72, 'Tue Dec 14 2021 12:23:18 GMT+0530 (India Standard Time)'), 18: (100, 'Wed Dec 01 2021 13:32:17 GMT+0530 (India Standard Time)'), 20: (100, 'Wed Dec 01 2021 13:32:18 GMT+0530 (India Standard Time)'), 11: (76, 'Fri Dec 03 2021 15:10:04 GMT+0530 (India Standard Time)'), 27: (73, 'Fri Dec 03 2021 15:10:09 GMT+0530 (India Standard Time)'), 1: (100, 'Fri Dec 03 2021 15:10:14 GMT+0530 (India Standard Time)'), 17: (70, 'Tue Dec 07 2021 12:28:41 GMT+0530 (India Standard Time)'), 23: (95, 'Tue Dec 07 2021 13:42:37 GMT+0530 (India Standard Time)'), 21: (100, 'Tue Dec 07 2021 13:42:43 GMT+0530 (India Standard Time)'), 22: (100, 'Tue Dec 07 2021 13:42:39 GMT+0530 (India Standard Time)'), 24: (90, 'Tue Dec 07 2021 13:42:49 GMT+0530 (India Standard Time)'), 19: (92, 'Fri Dec 10 2021 11:08:20 GMT+0530 (India Standard Time)'), 119: (93, 'Tue Dec 14 2021 12:23:18 GMT+0530 (India Standard Time)'), 121: (97, 

In [13]:
max_lessons = {
    'kn': 379,
    'en': 303,
    'maths': 404
}
# 1, 1.5, 2.5, 3
month = 4
total_months = 10
alpha_avg = 0
for student, course in students.items():
    course_avg = {
        'kn': 0,
        'en': 0,
        'maths': 0
    }
    lesson_details = []
    for course_name, lessons in course.items():
        if course_name in max_lessons:
            lesson_avg = 0
            score_avg = 0
            for lesson_order, score in lessons.items():
                lesson_avg += lesson_order
                score_avg += score[0]
            lesson_avg = lesson_avg / max(max_lessons[course_name]/total_months, len(lessons))
            ideal_lesson_avg = max_lessons[course_name] * (month - 0.5)/total_months
            score_avg /= len(lessons)
            # print(course_name, lesson_avg, ideal_lesson_avg, lesson_avg/ideal_lesson_avg, score_avg, len(lessons), max_lessons[course_name]/total_months)
            course_avg[course_name] = lesson_avg/ideal_lesson_avg * (score_avg + 15) / 100
            lesson_details.append({'course': course_name, 'lesson_num': lesson_order, 'score': score[0], 'date': score[1]})
    alpha = (course_avg['kn'] + course_avg['en'] + course_avg['maths']) / 3
    print(student, '\t', alpha, '\t', json.dumps(lesson_details))
    alpha_avg += alpha

KD07 	 0.354901256993618 	 [{"course": "kn", "lesson_num": 104, "score": 75, "date": "Tue Dec 28 2021 12:31:38 GMT+0530 (India Standard Time)"}, {"course": "en", "lesson_num": 42, "score": 80, "date": "Fri Dec 03 2021 15:09:44 GMT+0530 (India Standard Time)"}, {"course": "maths", "lesson_num": 5, "score": 100, "date": "Mon Dec 20 2021 12:54:27 GMT+0530 (India Standard Time)"}]
KD10 	 0.7558271305600455 	 [{"course": "kn", "lesson_num": 318, "score": 38, "date": "Thu Dec 30 2021 13:36:14 GMT+0530 (India Standard Time)"}, {"course": "en", "lesson_num": 186, "score": 45, "date": "Wed Dec 15 2021 10:45:27 GMT+0530 (India Standard Time)"}, {"course": "maths", "lesson_num": 99, "score": 67, "date": "Mon Dec 20 2021 12:30:47 GMT+0530 (India Standard Time)"}]
KD01 	 0.488968809145683 	 [{"course": "en", "lesson_num": 29, "score": 100, "date": "Tue Dec 14 2021 11:21:25 GMT+0530 (India Standard Time)"}, {"course": "maths", "lesson_num": 34, "score": 45, "date": "Thu Dec 16 2021 13:32:59 GMT+0530

In [10]:
print(alpha_avg)

148.73893800343026
